In [1]:
import torch 
import torch.nn as nn 
import torch.optim as optim 

import matplotlib.pyplot as plt
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

In [2]:
# get the training and test data 
train_data = datasets.FashionMNIST(root='DATA',train=True,download=True,transform=ToTensor(),target_transform=None)
test_data = datasets.FashionMNIST(root='DATA',train=False, download=True,transform=ToTensor())

In [3]:
# check input and output shapes 
train_data[0][0].shape

torch.Size([1, 28, 28])

In [4]:
# create data loaders 

batch_size = 32 

# these are iterables 
train_dataloader = DataLoader(train_data,batch_size=batch_size,shuffle=True)
test_dataloader = DataLoader(test_data,batch_size=batch_size,shuffle=False)

# length of data loader = len(data)/batch_size
len(train_dataloader), len(test_dataloader)

(1875, 313)

In [7]:
class FashionMNISTModelV2(nn.Module):
    def __init__(self,input_shape,output_shape,hidden_units):
        super().__init__()
        self.block_1 = nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels=hidden_units,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,out_channels=hidden_units,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2))
        
        self.block_2 = nn.Sequential(
            nn.Conv2d(hidden_units,hidden_units,3,padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_units,hidden_units,3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2))
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units*7*7, out_features=output_shape))
        
    def forward(self,x):
        return self.classifier(self.block_2(self.block_1(x)))

In [9]:
    
model_2 = FashionMNISTModelV2(input_shape=1,hidden_units=10,output_shape=10)
model_2

FashionMNISTModelV2(
  (block_1): Sequential(
    (0): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=490, out_features=10, bias=True)
  )
)

In [13]:
# loss/accuracy function and optimizer 

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_2.parameters(),lr=0.1)

# accuracy function 
def accuracy_fn(y_true,y_pred):
    correct = torch.eq(y_true,y_pred).sum().item()
    acc = (correct/len(y_pred)) * 100
    return acc

In [14]:
# train/test fucntions 

# train function 
def train_step(model, data_loader, loss_fn, optimizer,accuracy_fn, device):
    train_loss, train_acc = 0, 0 
    model.to(device)
    
    for batch, (X,y) in enumerate(data_loader):
        X,y = X.to(device),y.to(device) # send data to GPU 
        y_pred = model(X)               # forward pass 
        # losses 
        loss = loss_fn(y_pred,y)
        train_loss += loss 
        train_acc += accuracy_fn(y,y_pred.argmax(dim=1))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    train_loss/=len(data_loader)
    train_acc/=len(data_loader)
    
    print(f'Train loss:{train_loss:.5f} | Train accuracy:{train_acc:.2f} %')
    
        
# test function 
def test_step(model,data_loader,loss_fn,accuracy_fn,device):
    test_loss, test_acc = 0,0 
    model.to(device)
    model.eval()
    
    with torch.inference_mode():
        for X,y in data_loader:
            X,y = X.to(device),y.to(device)
            test_pred = model(X)
            test_loss += loss_fn(test_pred,y)
            test_acc += accuracy_fn(y,test_pred.argmax(dim=1))
            
        test_loss /= len(data_loader)
        test_acc/= len(data_loader)
    
        print(f'Test loss:{test_loss:.5f} | Test acc : {test_acc:.2f} %')
        

In [16]:
# training loop 
device = 'cuda' if torch.cuda.is_available() else 'cpu'
epochs = 3 
for epoch in range(epochs):
    print(f'Epoch : {epoch} \n..............')
    # train 
    train_step(model=model_2,data_loader=train_dataloader,loss_fn=loss_fn,accuracy_fn=accuracy_fn,optimizer=optimizer,device=device)
    # test 
    test_step(model=model_2,data_loader=train_dataloader,loss_fn=loss_fn,accuracy_fn=accuracy_fn,device=device)

Epoch : 0 
..............
Train loss:0.58554 | Train accuracy:78.71 %
Test loss:0.35178 | Test acc : 87.37 %
Epoch : 1 
..............
Train loss:0.34964 | Train accuracy:87.42 %
Test loss:0.35807 | Test acc : 86.93 %
Epoch : 2 
..............
Train loss:0.31538 | Train accuracy:88.55 %
Test loss:0.29749 | Test acc : 89.18 %
